# Hotel Booking Cancellations - EDA & Preprocessing

In [84]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Optional, List

sns.set_theme(style='whitegrid')

DATA_PATH = '../data/hotel_bookings.csv'

In [85]:
def load_hotel_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    print(f"Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

def get_basic_summary(df: pd.DataFrame) -> None:
    print("Dataset Summary:")
    display(df.info())
    print("\nMissing Values:")
    null_values = df.isnull().sum()
    display(null_values[null_values > 0])
    print("\nStatistical Summary:")
    display(df.describe())

    duplicates = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicates}")
    print(f"Percentage of duplicate rows: {duplicates / len(df) * 100:.2f}%")

    

In [86]:
df = load_hotel_data(DATA_PATH)
get_basic_summary(df)

Data loaded: 119390 rows, 32 columns
Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                  

None


Missing Values:


children         4
country        488
agent        16340
company     112593
dtype: int64


Statistical Summary:


,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


Number of duplicate rows: 31994
Percentage of duplicate rows: 26.80%


Meaning of columns with null values:
- children: Number of children. => Null values may indicate no children. => replace with 0?
- country: Country of origin (ISO format e.g., "POL", "ESP"). => ?
- agent: ID of the travel agent who made the booking. => Null values may indicate direct bookings. => replace with 0? or add new category 'direct'?
- company: ID of the company/entity that made the booking or responsible for paying the booking. => Null values may indicate individual bookings. => replace with 0? or add new category 'individual'?

Additionally based on dataset description on Kaggle
- meal: Undefined and SC is the same category (no meal package) => worth to change to the same value

In [87]:
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    fill_values = {
        'children': 0,
        'country': 'Unknown',
        'agent': 0,
        'company': 0
    }
    
    df = df.fillna(fill_values)
    
    # children type should be integer
    df['children'] = df['children'].astype(int)
    
    print("Handled missing values for columns:", ', '.join(fill_values.keys()))
    return df

In [88]:
def handle_inconsistent_values(df: pd.DataFrame) -> pd.DataFrame:
    # Fix meal naming inconsistency (Undefined and SC are the same based on kaggle description)
    df['meal'] = df['meal'].replace('Undefined', 'SC')

    print("Handled inconsistent values for columns: meal")
    return df

In [ ]:
def clean_invalid_records(df: pd.DataFrame) -> pd.DataFrame:
    initial_rows = len(df)
    
    # at least one guest should be present
    df = df[(df['adults'] + df['children'] + df['babies']) > 0]
    
    # ADR (Price) should be positive and not an extreme outlier (assuming max 1000 for sanity)
    df = df[(df['adr'] >= 0) & (df['adr'] <= 1000)]

    # impossible guest counts (assuming max 30 guests per booking as a reasonable limit)
    df = df[(df['adults'] + df['children'] + df['babies']) <= 30]
    
    print(f"Cleaned invalid records. Dropped {initial_rows - len(df)} rows.")
    return df


In [90]:
# Drop duplicates? - might change the prediction depending on the model used, so maybe decide on each model training separately - there is no booking_id, so duplicates might just have the same features but be different bookings
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    initial_rows = len(df)
    df = df.drop_duplicates()
    print(f"Removed {initial_rows - len(df)} duplicate rows.")
    return df

In [ ]:
# these columns are not available before checkout / cancellation, so they would leak information about the target
def remove_leakage_columns(df: pd.DataFrame) -> pd.DataFrame:
    leakage_columns = ['reservation_status', 'reservation_status_date']
    df = df.drop(columns=leakage_columns)
    print(f"Removed leakage columns: {', '.join(leakage_columns)}")
    return df

In [92]:
def add_new_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
    df['total_guests'] = df['adults'] + df['children'] + df['babies']
    df['is_family'] = ((df['children'] > 0) | (df['babies'] > 0)).astype(int)

    print(f"Feature engineering complete. Current shape: {df.shape}")
    return df

In [93]:
def save_data(df: pd.DataFrame, output_path: str) -> None:
    df.to_csv(output_path, index=False)
    print(f"Processed data successfully saved to: {output_path}")

In [94]:
raw_df = load_hotel_data(DATA_PATH)

processed_df = (raw_df
                .pipe(handle_missing_values)
                .pipe(handle_inconsistent_values)
                .pipe(clean_invalid_records)
                # .pipe(remove_duplicates) TODO: decide on duplicates handling
                .pipe(remove_leakage_columns)
                .pipe(add_new_features))

save_data(processed_df, '../data/hotel_bookings_cleaned.csv')
get_basic_summary(processed_df)

Data loaded: 119390 rows, 32 columns
Handled missing values for columns: children, country, agent, company
Handled inconsistent values for columns: meal
Cleaned invalid records. Dropped 185 rows.
Removed leakage columns: reservation_status, reservation_status_date
Feature engineering complete. Current shape: (119205, 33)
Processed data successfully saved to: ../data/hotel_bookings_cleaned.csv
Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 119205 entries, 0 to 119389
Data columns (total 33 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119205 non-null  object 
 1   is_canceled                     119205 non-null  int64  
 2   lead_time                       119205 non-null  int64  
 3   arrival_date_year               119205 non-null  int64  
 4   arrival_date_month              119205 non-null  object 
 5   arrival_date_week_number        119205 non

None


Missing Values:


Series([], dtype: int64)


Statistical Summary:


,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,total_nights,total_guests,is_family
count,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,...,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000,119205.000000
mean,0.370748,104.103460,2016.156495,27.163366,15.799010,0.927033,2.499199,1.858034,0.104048,0.007961,...,0.218783,74.889828,10.735850,2.321312,101.928122,0.062556,0.571528,3.426232,1.970043,0.078285
std,0.483007,106.871354,0.707472,13.601127,8.780976,0.995086,1.897099,0.524895,0.398843,0.097511,...,0.638492,107.168782,53.831227,17.598365,48.040218,0.245364,0.792884,2.540661,0.679507,0.268621
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,...,0.000000,7.000000,0.000000,0.000000,69.500000,0.000000,0.000000,2.000000,2.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,...,0.000000,9.000000,0.000000,0.000000,94.950000,0.000000,0.000000,3.000000,2.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,...,0.000000,152.000000,0.000000,0.000000,126.000000,0.000000,1.000000,4.000000,2.000000,0.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,27.000000,10.000000,10.000000,...,18.000000,535.000000,543.000000,391.000000,510.000000,8.000000,5.000000,69.000000,27.000000,1.000000


Number of duplicate rows: 32238
Percentage of duplicate rows: 27.04%


# TO BE DECIDED:
- What to do with duplicates?
- What to do with df_cancelled['reserved_room_type'] != df_cancelled['assigned_room_type']? Is this data leakage??
```
find if out of these bookings, there are any with 'reserved_room_type' different than 'assigned_room_type' => this might be a data leakage, because the room rarely changes for cancelled bookings
leakage_cases = df_cancelled[df_cancelled['reserved_room_type'] != df_cancelled['assigned_room_type']]
print(f"Number of changed rooms out of cancelled bookings: {len(leakage_cases)}")
print(f"Percentage of changed rooms out of cancelled bookings: {len(leakage_cases) / len(df_cancelled) * 100:.2f}%")
print(f"Percentage of changed rooms out of all bookings: {len(df[df['reserved_room_type'] != df['assigned_room_type']]) / len(df) * 100:.2f}%")
print("-----")
```